In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    str_cols = ""
    for col in df.columns[:-1]:
        str_cols += col + "\t"
    str_cols += df.columns.values[-1]+ "\n"
    f.write(str(str_cols))
    for row in df.values:
        str_row = ""
        for col in row[:-1]:
            str_row += str(col) + "\t"
        str_row += str(row[-1])+ "\n"
        f.write(str_row)
    f.close

In [3]:
gvs_chr20_read= "/home/andre/Documents/trabalho/datasets/Genomic_variants/chr20/allChr20.GRCH38.sorted.vcf"
gvs_chr20_df = pd.read_csv(gvs_chr20_read, sep="\t", usecols=['#CHROM',  'POS', 'REF', 'ALT', 'INFO'])

gvs_chr20_df

,#CHROM,POS,REF,ALT,INFO
0,20,60114,T,C,AC=1;AN=5096;DP=67920;AF=0;EAS_AF=0;EUR_AF=0;A...
1,20,60138,T,A,AC=1;AN=5096;DP=69096;AF=0;EAS_AF=0;EUR_AF=0;A...
2,20,60149,C,T,AC=3;AN=5096;DP=66846;AF=0;EAS_AF=0;EUR_AF=0;A...
3,20,60424,A,C,AC=0;AN=5096;DP=29861;AF=0;EAS_AF=0;EUR_AF=0;A...
4,20,60440,A,T,AC=2;AN=5096;DP=26998;AF=0;EAS_AF=0;EUR_AF=0;A...
...,...,...,...,...,...
1817487,20,64333367,G,A,AC=6;AN=5096;DP=16236;AF=0;EAS_AF=0;EUR_AF=0;A...
1817488,20,64333468,C,T,AC=1;AN=5096;DP=13363;AF=0;EAS_AF=0;EUR_AF=0;A...
1817489,20,64333615,G,A,AC=16;AN=5096;DP=13225;AF=0;EAS_AF=0;EUR_AF=0;...
1817490,20,64333621,G,A,AC=10;AN=5096;DP=13129;AF=0;EAS_AF=0;EUR_AF=0;...


In [4]:
#writeNewtxt(gvs_chr20_read, gvs_chr20_df)

In [5]:
#gvs_chr20_df["INFO"][0]
#gvs_chr20_df["INFO"][30]

### Dividir as genomic variants em SNPs e INDELs

In [6]:
#snps_chr20_df = get_df_snp(gvs_chr20_df)
#snps_chr20_df
#mask1 = (gvs_chr20_df['REF'].str.len() == 1) & (gvs_chr20_df['ALT'].str.len() == 1)
#snps_chr20_df = gvs_chr20_df.loc[mask1]
#snps_chr20_df

In [7]:
#indels_chr20_df = get_df_indels(gvs_chr20_df)
#indels_chr20_df
#mask2 = (gvs_chr20_df['REF'].str.len() > 1) | (gvs_chr20_df['ALT'].str.len() > 1)
#indels_chr20_df = gvs_chr20_df.loc[mask2]
#indels_chr20_df

## Seperated density values for insertions and deletions

In [8]:
#inserts_chr20 = indels_chr20_df.loc[(indels_chr20_df['REF']) < (indels_chr20_df['ALT'])]
#inserts_chr20

In [9]:
#deletions_chr20 = indels_chr20_df.loc[(indels_chr20_df['REF']) > (indels_chr20_df['ALT'])]
#deletions_chr20

## Funções de calculo de densidades

In [4]:
def calc_snpDensity(cb_df_chr, gv_df_chr):#, chrom = "chr20"):
    """calculates the density of snps within the cytobands of one chromosome
    """
    densities = []
    #chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
    chr_size = cb_df_chr["Last_index"][cb_df_chr.index[-1]]+1
    chr_gvs = np.zeros((chr_size), dtype=int)
    #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
    for row in gv_df_chr.index:
        chr_gvs[gv_df_chr["POS"][row]]+=1
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        cytoband_array = chr_gvs[int(cb_df_chr["First_index"][cytoband]):int(cb_df_chr["Last_index"][cytoband])+1]
        snp_indel_length = np.count_nonzero(cytoband_array)
        cytoband_length = cb_df_chr["Size"][cytoband]
        density = snp_indel_length/cytoband_length
        densities.append(round(density, 7))
        
    cb_df_chr.insert(len(cb_df_chr.columns), "SNP_density", densities, True)
    return cb_df_chr

In [5]:
def calc_indelDens(cb_df_chr, gv_df_chr):#, chrom = "chr20"):
    """calculates the density of indels within the cytobands of one chromosome
    """
    densities = []
    #chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
    chr_size = cb_df_chr["Last_index"][cb_df_chr.index[-1]]+1
    chr_gvs = np.zeros((chr_size), dtype=int)
    #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
    for row in gv_df_chr.index:
        if len(gv_df_chr["REF"][row]) > len(gv_df_chr["ALT"][row]):               #Deletion
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["REF"][row])+1]+=1
        else:                                                                    #Insertion
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["ALT"][row])+1]+=1
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        cytoband_array = chr_gvs[int(cb_df_chr["First_index"][cytoband]):int(cb_df_chr["Last_index"][cytoband])+1]
        snp_indel_length = np.count_nonzero(cytoband_array)
        cytoband_length = cb_df_chr["Size"][cytoband]
        density = snp_indel_length/cytoband_length
        densities.append(round(density, 7))
        
    cb_df_chr.insert(len(cb_df_chr.columns), "Indel_density", densities, True)
    return cb_df_chr

In [6]:
def calc_insert_deleteDens(cb_df_chr, gv_df_chr, dens_type = "Inserts_density"):#, chrom = "chr20"):
    """calculates the density of inserts or deletes within the cytobands of one chromosome
    """
    densities = []
    #chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == chrom]
    chr_size = cb_df_chr["Last_index"][cb_df_chr.index[-1]]+1
    chr_gvs = np.zeros((chr_size), dtype=int)
    #percorrer o dataset pelas vulnerabilidades no cromossoma e anotar na lista np.zeroes
    for row in gv_df_chr.index:
        if dens_type == "Inserts_density":
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["ALT"][row])+1]+=1
        elif dens_type == "Deletes_density":
            chr_gvs[gv_df_chr["POS"][row]:gv_df_chr["POS"][row] + len(gv_df_chr["REF"][row])+1]+=1
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        cytoband_array = chr_gvs[int(cb_df_chr["First_index"][cytoband]):int(cb_df_chr["Last_index"][cytoband])+1]
        snp_indel_length = np.count_nonzero(cytoband_array)
        cytoband_length = cb_df_chr["Size"][cytoband]
        density = snp_indel_length/cytoband_length
        densities.append(round(density, 10))
        
    cb_df_chr.insert(len(cb_df_chr.columns), dens_type, densities, True)
    return cb_df_chr

In [7]:
def insert_0sColumn(cb_df_chr):#, chrom = "chr20"):
    """calculates the density of snps within the cytobands of one chromosome
    """
    densities = []
    #Calculo da densidade para cada cytoband
    for cytoband in cb_df_chr.index:
        #densities.append(round(0.000000, 10))
        densities.append(np.nan)
    cb_df_chr.insert(len(cb_df_chr.columns), "SNP_density", densities, True)
    cb_df_chr.insert(len(cb_df_chr.columns), "Indel_density", densities, True)
    cb_df_chr.insert(len(cb_df_chr.columns), "Inserts_density", densities, True)
    cb_df_chr.insert(len(cb_df_chr.columns), "Deletes_density", densities, True)
    return cb_df_chr

In [8]:
cytoPath_read = "/home/andre/Documents/trabalho/cytobands/cytobandFiltered_processed_str_dr_dg.txt"
cytoPath_write = "/home/andre/Documents/trabalho/cytobands/cytobandFiltered_processed_str_dr_dg_gv.txt"

cytobands_df = pd.read_csv(cytoPath_read, sep="\t")#, header=None))

cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,TandemRepeats_dens,STRs_dens,LTRs_dens,DRs_density,CDSs_density,CDSIntron_density,CDSExon_density,five_prime_UTRs_density,five_prime_UTRIntron_density,five_prime_UTRExon_density,three_prime_UTRs_density,three_prime_UTRIntron_density,three_prime_UTRExon_density,exons_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.087516,0.006384,0.084476,0.326328,0.016467,0.013392,0.003076,0.010664,0.005135,0.005529,0.026310,0.025853,0.000456,0.133749
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084077,0.013902,0.079223,0.281556,0.006066,0.004490,0.001576,0.003274,0.001761,0.001512,0.019776,0.019406,0.000370,0.060012
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056236,0.012909,0.051421,0.275410,0.010266,0.006903,0.003362,0.005659,0.002073,0.003586,0.025848,0.025715,0.000133,0.056062
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049082,0.013229,0.042217,0.404844,0.005996,0.004821,0.001175,0.003569,0.002070,0.001499,0.015875,0.015746,0.000129,0.034280
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.055881,0.011072,0.048928,0.404047,0.009799,0.006866,0.002934,0.004954,0.002176,0.002777,0.029553,0.028516,0.001037,0.066018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.036956,0.014211,0.029356,0.015158,0.000917,0.000627,0.000290,0.000927,0.000344,0.000583,0.002534,0.002459,0.000075,0.009953
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.061662,0.015902,0.052790,0.012965,0.002263,0.002263,0.000000,0.000346,0.000338,0.000008,0.000937,0.000635,0.000302,0.033020
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082234,0.005789,0.078347,0.006441,0.001096,0.000802,0.000294,0.000540,0.000037,0.000503,0.002733,0.002733,0.000000,0.021283
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026278,0.009164,0.020721,0.001822,0.001369,0.001294,0.000075,0.000652,0.000345,0.000308,0.002016,0.001749,0.000267,0.022278


### Cytoband data frame only chr20

In [15]:
#chr20_df = cytobands_df.loc[cytobands_df['Chromossome'] == "chr20"]

### Calculate the different densities

In [16]:
#chr20_df = calc_snpDensity(chr20_df, snps_chr20_df)

In [17]:
#chr20_df = calc_indelDens(chr20_df, indels_chr20_df)

In [18]:
#chr20_df = calc_insert_deleteDens(chr20_df, inserts_chr20, "Insertion_density")

In [19]:
#chr20_df = calc_insert_deleteDens(chr20_df, deletions_chr20, "Deletion_density")

In [20]:
#chr20_df

### Make plots of the data

In [21]:
def plot_1chr(chr_df, chromossome, col = "SNP_density"):
    plt.figure(figsize=(30,10))
    plt.plot(chr_df["Citoband"], chr_df[col], 'ro--', linewidth=2, markersize=6)
    plt.xlabel("Cytoband")
    plt.ylabel(col)
    plt.grid()
    plt.title(col + " in chromosome "  + chromossome)
    #plt.show()
    plt.savefig(chromossome + col)

In [22]:
#plot_1chr(chr20_df, "20", "SNP_density")

In [23]:
#plot_1chr(chr20_df, "20", "Indel_density")

In [24]:
#cytoPath_write = "/home/androx/Documents/trabalho/citobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_chr20.txt"
#writeNewtxt(cytoPath_write, chr20_df)

# Codigo de execução para cada um dos chrs

In [9]:
def execute_1chr(chromosome):
    """
    """
    gvs_chr_read= "/home/andre/Documents/trabalho/datasets/Genomic_variants/" + chromosome + "/" + chromosome + "_gvs.vcf"
    cytoband_write = "/home/andre/Documents/trabalho/cytobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_" + chromosome + ".txt"
    
    #print(gvs_chr_read)
    gvsChr_df = pd.read_csv(gvs_chr_read, sep="\t")           #genomic variants dataset

    ####### Separation of gv dataset into snps, indels, inserts and delets datasets ############
    mask1 = (gvsChr_df['REF'].str.len() == 1) & (gvsChr_df['ALT'].str.len() == 1)
    mask2 = (gvsChr_df['REF'].str.len() > 1) | (gvsChr_df['ALT'].str.len() > 1)
    
    snps_chr_df = gvsChr_df.loc[mask1]
    indels_chr_df = gvsChr_df.loc[mask2]
    inserts_chr_df = indels_chr_df.loc[(indels_chr_df['REF']) < (indels_chr_df['ALT'])]
    deletes_chr_df = indels_chr_df.loc[(indels_chr_df['REF']) > (indels_chr_df['ALT'])]

    #print(indels_chr_df)
    ####### Calculation of densities into the cytoband dataset
    chr_df = cytobands_df.loc[cytobands_df['Chromossome'] == chromosome]    #cytobands dataset of intended chromosome
    
    chr_df = calc_snpDensity(chr_df, snps_chr_df)
    chr_df = calc_indelDens(chr_df, indels_chr_df)
    chr_df = calc_insert_deleteDens(chr_df, inserts_chr_df, "Inserts_density")
    chr_df = calc_insert_deleteDens(chr_df, deletes_chr_df, "Deletes_density")
    print(chr_df)

    writeNewtxt(cytoband_write, chr_df)

In [22]:
execute_1chr("chrX")

    Chromossome  First_index  Last_index Citoband   Unkown     Size  \
812        chrX            0     4399999   p22.33     gneg  4399999   
813        chrX      4400000     6099999   p22.32   gpos50  1699999   
814        chrX      6100000     9599999   p22.31     gneg  3499999   
815        chrX      9600000    17399999    p22.2   gpos50  7799999   
816        chrX     17400000    19199999   p22.13     gneg  1799999   
817        chrX     19200000    21899999   p22.12   gpos50  2699999   
818        chrX     21900000    24899999   p22.11     gneg  2999999   
819        chrX     24900000    29299999    p21.3  gpos100  4399999   
820        chrX     29300000    31499999    p21.2     gneg  2199999   
821        chrX     31500000    37799999    p21.1  gpos100  6299999   
822        chrX     37800000    42499999    p11.4     gneg  4699999   
823        chrX     42500000    47599999    p11.3   gpos75  5099999   
824        chrX     47600000    50099999   p11.23     gneg  2499999   
825   

In [23]:
def add_cols_chrY():
    cytoband_write = "/home/andre/Documents/trabalho/cytobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_chrY.txt"
    chr_df = cytobands_df.loc[cytobands_df['Chromossome'] == "chrY"]

    chr_df = insert_0sColumn(chr_df)
    
    #return(chr_df)
    writeNewtxt(cytoband_write, chr_df)

In [24]:
add_cols_chrY()

### Juntar todos os dataframes criados cromossoma a cromossoma

In [25]:
def concat_dfs():
    frames = []
    chroms = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14", "chr15", "chr16", "chr17", "chr18", "chr19", "chr20", "chr21", "chr22", "chrX", "chrY"]
    for chromosome in chroms:
        gvs_chr_read= "/home/andre/Documents/trabalho/cytobands/Latest_gvs/cytobandFiltered_processed_str_dr_dg_gv_" + chromosome + ".txt"
        chr_df = pd.read_csv(gvs_chr_read, sep="\t")
        frames.append(chr_df)
    result = pd.concat(frames)
    return result

In [26]:
cb_df_sorted = concat_dfs()

In [28]:
cb_df_sorted = cb_df_sorted.reset_index(drop=True)

In [29]:
cb_df_sorted

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,TandemRepeats_dens,STRs_dens,LTRs_dens,DRs_density,...,five_prime_UTRIntron_density,five_prime_UTRExon_density,three_prime_UTRs_density,three_prime_UTRIntron_density,three_prime_UTRExon_density,exons_density,SNP_density,Indel_density,Inserts_density,Deletes_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.087516,0.006384,0.084476,0.326328,...,0.005135,0.005529,0.026310,0.025853,0.000456,0.133749,0.022000,0.006463,0.001590,0.004900
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084077,0.013902,0.079223,0.281556,...,0.001761,0.001512,0.019776,0.019406,0.000370,0.060012,0.033405,0.008639,0.002013,0.006659
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056236,0.012909,0.051421,0.275410,...,0.002073,0.003586,0.025848,0.025715,0.000133,0.056062,0.030704,0.009019,0.002377,0.006688
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049082,0.013229,0.042217,0.404844,...,0.002070,0.001499,0.015875,0.015746,0.000129,0.034280,0.028061,0.009503,0.002497,0.007044
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.055881,0.011072,0.048928,0.404047,...,0.002176,0.002777,0.029553,0.028516,0.001037,0.066018,0.028159,0.008768,0.002322,0.006476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,chrY,12400000,17099999,q11.221,gpos50,4699999,0.036956,0.014211,0.029356,0.015158,...,0.000344,0.000583,0.002534,0.002459,0.000075,0.009953,0.000000,0.000000,0.000000,0.000000
858,chrY,17100000,19599999,q11.222,gneg,2499999,0.061662,0.015902,0.052790,0.012965,...,0.000338,0.000008,0.000937,0.000635,0.000302,0.033020,0.000000,0.000000,0.000000,0.000000
859,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082234,0.005789,0.078347,0.006441,...,0.000037,0.000503,0.002733,0.002733,0.000000,0.021283,0.000000,0.000000,0.000000,0.000000
860,chrY,23800000,26599999,q11.23,gneg,2799999,0.026278,0.009164,0.020721,0.001822,...,0.000345,0.000308,0.002016,0.001749,0.000267,0.022278,0.000000,0.000000,0.000000,0.000000


In [32]:
file_write = "/home/andre/Documents/trabalho/cytobands/cytobandFiltered_processed_str_dr_dg_gv_sorted.txt"

In [34]:
writeNewtxt(file_write, cb_df_sorted)